# Quality Assurance & Control - GeoDjango Objects
Helps assure that the SQLite tables are spatially enabled, i.e. SpatiaLite, and are properly modeled by GeoDjango. This notebook also documents a lot of the validation needed to configure GeoDjango components.

### Configuring SpatiaLite and enabling GeoDjango
Within `settings.py` the following need to be configured:
- "GDAL_LIBRARY_PATH"
- "SPATIALITE_LIBRARY_PATH"
Also within `settings.py` "INSTALLED_APPS" needs to be amended to include `'django.contrib.gis'` and the "DATABASES" 'ENGINE' needs to be amended to be `'django.contrib.gis.db.backends.spatialite'`.

At `[PROJECT]/` you need to download and unzip SpatiaLite from [here](https://www.gaia-gis.it/gaia-sins/) as well as make the amendments to your `libgdal.py` recommended in [this](https://stackoverflow.com/questions/46313119/geodjango-could-not-find-gdal-library-in-windows-10) StackOverflow post.

### Spatially Enabling Django Database
Use the following code as a template to spatially enable your Django SQLite database turning it into a SpatiaLite database and therefore making your application a GeoDjango application. This code builds the metadata tables for spatial data.

```python
import sqlite3

db = "path/to/django/database/db.sqlite3"

conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()
c.execute('''SELECT InitSpatialMetaData();''')
conn.commit()
conn.close()
```

### Validating SpatiaLite and GeoDjango
From an Anaconda prompt window, activate your environment of choice and nagivate to your GeoDjango database. Here issue a command something like `sqlite3 db.sqlite3` which will turn your Anaconda prompt into a sqlite terminal. From here issue the command `.tables` you should see a bunch of geometry tables among others. Quit this termainl with `.quit`.

From the Anaconda prompt, activate a GeoDjango Database Shell terminal with `python manage.py dbshell` this should launch you into a spatialite terminal. Quit out of this termainl with `.quit`.

From the Anaconda prompt, activate a GeoDjango Shell terminal with `python manage.py shell`. At this point, we just want to make sure this starts up and will validate what should come out of here below since we are doing commands here, in the Jupyter Notebook, that could otherwise be implimented within the Shell terminal.

### Import some libraries, configure Django
Since we are within Jupyter Notebook, some stuff has to be configured specifically to handle this development environment.

In [1]:
# Basic stack
import os

# Geospatial stack
import shapely
from shapely.wkt import loads
from shapely.geometry import Polygon
import pandas as pd
import geopandas as gpd
import folium

# GeoDjango stack
import sqlite3
import django
from django.db import connection
from django.contrib.gis.geos import GEOSGeometry
from asgiref.sync import sync_to_async

import sys; sys.path.append('../../')
os.environ['DJANGO_SETTINGS_MODULE'] = 'gaia.settings'
django.setup()

from whale.models import AreaOfInterest as AOI
from whale.models import ExtractTransformLoad as ETL

### User defined variables

In [2]:
db = "../../db.sqlite3"
whale_earthexplorer_columns = ['entity_id', 'catalog_id', 'acquisition_date', 'vendor',
                               'vendor_id', 'cloud_cover', 'satellite', 'sensor',
                               'number_of_bands', 'map_projection', 'datum',
                               'processing_level', 'file_format', 'license_id',
                               'sun_azimuth', 'sun_elevation', 'pixel_size_x',
                               'pixel_size_y', 'license_uplift_update', 'event',
                               'date_entered', 'center_latitude_dec',
                               'center_longitude_dec', 'thumbnail', 'publish_date',
                               'aoi_id_id', 'event_date']
whale_areaofinterest_columns = ['id', 'name', 'requestor', 'sqkm']
etl_columns = ['table_name', 'aoi_id', 'id', 'vendor_id', 'entity_id', 'vendor',
               'platform', 'pixel_size_x', 'pixel_size_y', 'date', 'publish_date',
               'sea_state_qual', 'sea_state_quant', 'shareable']

### Validate geospatial data within the SpatiaLite database

In [3]:
# Connect to database, load SpatiaLite exntention
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

# Using the column names within Whale's EarthExplorer table, select first ten records
columns_str = ', '.join(whale_earthexplorer_columns)
sql_string = "SELECT {}, AsText(bounds) FROM whale_earthexplorer LIMIT 10".format(columns_str)
df = pd.read_sql_query(sql_string, conn)

# Build GeoDataFrame from results
df = df.rename(columns={'AsText(bounds)': 'geometry'}, errors='raise')
df['geometry'] = shapely.wkt.loads(df['geometry'])
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Close your database connection
conn.commit()
conn.close()

# Show records
gdf.head()

,entity_id,catalog_id,acquisition_date,vendor,vendor_id,cloud_cover,satellite,sensor,number_of_bands,map_projection,...,license_uplift_update,event,date_entered,center_latitude_dec,center_longitude_dec,thumbnail,publish_date,aoi_id_id,event_date,geometry
0,WV320240510151159M00,10400100959B5400,2024-05-10,MAXAR TECHNOLOGIES,24MAY10151159-M1BS-508496072030_01_P001,76,WORLDVIEW-3,MSI,8,GCP,...,None,UCDAM,2024-05-21,41.867831,-70.227235,https://ims.cr.usgs.gov/thumbnail/CRSSP/WV/202...,2024-05-21 13:51:35.011890,6,None,"POLYGON ((-70.36047 41.80772, -70.36047 41.927..."
1,WV320240510151159P00,10400100959B5400,2024-05-10,MAXAR TECHNOLOGIES,24MAY10151159-P1BS-508496072030_01_P001,76,WORLDVIEW-3,PAN,1,GCP,...,None,UCDAM,2024-05-21,41.867839,-70.227219,https://ims.cr.usgs.gov/thumbnail/CRSSP/WV/202...,2024-05-21 13:51:36.918914,6,None,"POLYGON ((-70.36013 41.80775, -70.36013 41.927..."
2,WV320240510151200M00,10400100959B5400,2024-05-10,MAXAR TECHNOLOGIES,24MAY10151200-M1BS-508496072030_01_P002,50,WORLDVIEW-3,MSI,8,GCP,...,None,UCDAM,2024-05-21,41.932398,-70.227268,https://ims.cr.usgs.gov/thumbnail/CRSSP/WV/202...,2024-05-21 13:52:06.426599,6,None,"POLYGON ((-70.36083 41.87082, -70.36083 41.993..."
3,WV320240510151200P00,10400100959B5400,2024-05-10,MAXAR TECHNOLOGIES,24MAY10151200-P1BS-508496072030_01_P002,50,WORLDVIEW-3,PAN,1,GCP,...,None,UCDAM,2024-05-21,41.932406,-70.227252,https://ims.cr.usgs.gov/thumbnail/CRSSP/WV/202...,2024-05-21 13:51:38.676543,6,None,"POLYGON ((-70.36050 41.87085, -70.36050 41.993..."
4,WV320240510151201M00,10400100959B5400,2024-05-10,MAXAR TECHNOLOGIES,24MAY10151201-M1BS-508496072030_01_P003,23,WORLDVIEW-3,MSI,8,GCP,...,None,UCDAM,2024-05-21,41.997315,-70.227204,https://ims.cr.usgs.gov/thumbnail/CRSSP/WV/202...,2024-05-21 13:51:37.779161,6,None,"POLYGON ((-70.36117 41.93425, -70.36117 42.060..."


### Plot GeoDataFrame on an interactive map

In [4]:
def style_function(hex_value):
    return {'color': hex_value, 'fillOpacity': 0}

# Add OpenStreetMap as a basemap
map = folium.Map()
folium.TileLayer('openstreetmap').add_to(map)

# Create a GeoJson layer from the response_geojson and add it to the map
folium.GeoJson(
    gdf['geometry'].to_json(),
    style_function = lambda x: style_function('#0000FF')
).add_to(map)

# Zoom to collected images
map.fit_bounds(map.get_bounds(), padding=(100, 100))

# Display the map
map

### Leverage GeoDjango's models to retrieve database objects
GeoDjango's `models.py` file provides GeoDjango with a blueprint for how the application should interact with the database returning objects. This means datatypes, constraints, etc. need to match between the models and the database. If not, there is no guarantee the application will retrieve the information we desire if any information at all.

Above, note that we used the table "whale_earthexplorer" where the application name is appended as a prefix to the model class. To validate that we can also retrieve the same information as objects let's import EarthExplorer from the Whale models and retrieve as well as plot this information.

In [5]:
from whale.models import EarthExplorer as EE

# Retrieve the first ten EarthExplorer objects from GeoDjango
imgs = await sync_to_async(list)(EE.objects.all()[:10])

# Create a GeoDataFrame
geoms = []
attributes = []
for img in imgs:
    attr_dict = {col: getattr(img, col) for col in whale_earthexplorer_columns}
    attributes.append(attr_dict)

    geoms.append(GEOSGeometry(img.bounds))

gdf = gpd.GeoDataFrame(attributes, geometry = [loads(g.wkt) for g in geoms])
gdf.head()

,entity_id,catalog_id,acquisition_date,vendor,vendor_id,cloud_cover,satellite,sensor,number_of_bands,map_projection,...,license_uplift_update,event,date_entered,center_latitude_dec,center_longitude_dec,thumbnail,publish_date,aoi_id_id,event_date,geometry
0,WV320240510151159M00,10400100959B5400,2024-05-10,MAXAR TECHNOLOGIES,24MAY10151159-M1BS-508496072030_01_P001,76,WORLDVIEW-3,MSI,8,GCP,...,None,UCDAM,2024-05-21,41.867831,-70.227235,https://ims.cr.usgs.gov/thumbnail/CRSSP/WV/202...,2024-05-21 13:51:35.011890+00:00,6,None,"POLYGON ((-70.36047 41.80772, -70.36047 41.927..."
1,WV320240510151159P00,10400100959B5400,2024-05-10,MAXAR TECHNOLOGIES,24MAY10151159-P1BS-508496072030_01_P001,76,WORLDVIEW-3,PAN,1,GCP,...,None,UCDAM,2024-05-21,41.867839,-70.227219,https://ims.cr.usgs.gov/thumbnail/CRSSP/WV/202...,2024-05-21 13:51:36.918914+00:00,6,None,"POLYGON ((-70.36013 41.80775, -70.36013 41.927..."
2,WV320240510151200M00,10400100959B5400,2024-05-10,MAXAR TECHNOLOGIES,24MAY10151200-M1BS-508496072030_01_P002,50,WORLDVIEW-3,MSI,8,GCP,...,None,UCDAM,2024-05-21,41.932398,-70.227268,https://ims.cr.usgs.gov/thumbnail/CRSSP/WV/202...,2024-05-21 13:52:06.426599+00:00,6,None,"POLYGON ((-70.36083 41.87082, -70.36083 41.993..."
3,WV320240510151200P00,10400100959B5400,2024-05-10,MAXAR TECHNOLOGIES,24MAY10151200-P1BS-508496072030_01_P002,50,WORLDVIEW-3,PAN,1,GCP,...,None,UCDAM,2024-05-21,41.932406,-70.227252,https://ims.cr.usgs.gov/thumbnail/CRSSP/WV/202...,2024-05-21 13:51:38.676543+00:00,6,None,"POLYGON ((-70.36050 41.87085, -70.36050 41.993..."
4,WV320240510151201M00,10400100959B5400,2024-05-10,MAXAR TECHNOLOGIES,24MAY10151201-M1BS-508496072030_01_P003,23,WORLDVIEW-3,MSI,8,GCP,...,None,UCDAM,2024-05-21,41.997315,-70.227204,https://ims.cr.usgs.gov/thumbnail/CRSSP/WV/202...,2024-05-21 13:51:37.779161+00:00,6,None,"POLYGON ((-70.36117 41.93425, -70.36117 42.060..."


### Plot the results

In [6]:
def style_function(hex_value):
    return {'color': hex_value, 'fillOpacity': 0}

# Add OpenStreetMap as a basemap
map = folium.Map()
folium.TileLayer('openstreetmap').add_to(map)

# Create a GeoJson layer from the response_geojson and add it to the map
folium.GeoJson(
    gdf['geometry'].to_json(),
    style_function = lambda x: style_function('#0000FF')
).add_to(map)

# Zoom to collected images
map.fit_bounds(map.get_bounds(), padding=(100, 100))

# Display the map
map

### Let's confirm the above code works for other models like AOI

In [7]:
# Retrieve the first ten EarthExplorer objects from GeoDjango
imgs = await sync_to_async(list)(AOI.objects.all()[:10])

# Create a GeoDataFrame
geoms = []
attributes = []
for img in imgs:
    attr_dict = {col: getattr(img, col) for col in whale_areaofinterest_columns}
    attributes.append(attr_dict)

    geoms.append(GEOSGeometry(img.geometry))

gdf = gpd.GeoDataFrame(attributes, geometry = [loads(g.wkt) for g in geoms])
gdf.head()

,id,name,requestor,sqkm,geometry
0,1,Kenai,,416.53,"POLYGON ((-151.59303 60.32094, -151.43153 60.6..."
1,2,Kalgin Island,,884.65,"POLYGON ((-151.73795 60.71243, -151.69676 60.5..."
2,3,Trading Bay,,529.37,"POLYGON ((-151.34361 61.01015, -151.22669 60.9..."
3,4,Tuxedni,,438.72,"POLYGON ((-152.60081 60.02992, -152.59393 60.0..."
4,5,Upper CI,,1871.28,"POLYGON ((-150.67303 60.95571, -151.00321 61.1..."


### Plot the results

In [8]:
def style_function(hex_value):
    return {'color': hex_value, 'fillOpacity': 0}

# Add OpenStreetMap as a basemap
map = folium.Map()
folium.TileLayer('openstreetmap').add_to(map)

# Create a GeoJson layer from the response_geojson and add it to the map
folium.GeoJson(
    gdf['geometry'].to_json(),
    style_function = lambda x: style_function('#0000FF')
).add_to(map)

# Zoom to collected images
map.fit_bounds(map.get_bounds(), padding=(100, 100))

# Display the map
map

### Now, let's repeat this for the ExtractTransformLoad (ETL) Table
Note that the ETL table is a table created from other table. I've seen this described as a junction or denormalized table amongst others. The key point is that this table integrates imagery across our three data sources (USGS EarthExplorer, GEGD, and Maxar Geospatial Platform) into a uniform table. As such, you'll notice there is no "whale_" prefix. Records are added to the table from triggers. This is the table we want to ensure always passes QA/QC.

In [10]:
# Retrieve the first 25 EarthExplorer objects from GeoDjango
imgs = await sync_to_async(list)(ETL.objects.all()[:25])

# Create a GeoDataFrame
geoms = []
attributes = []
for img in imgs:
    attr_dict = {col: getattr(img, col) for col in etl_columns}
    attributes.append(attr_dict)

    geoms.append(GEOSGeometry(img.geometry))

gdf = gpd.GeoDataFrame(attributes, geometry = [loads(g.wkt) for g in geoms])
gdf.head(25)

,table_name,aoi_id,id,vendor_id,entity_id,vendor,platform,pixel_size_x,pixel_size_y,date,publish_date,sea_state_qual,sea_state_quant,shareable,geometry
0,EE,6,104001003D6A7700,18MAY24154658-M1BS-503577655070_01_P001,WV320180524154658M00,DIGITAL GLOBE,WORLDVIEW-3,1,1,2018-05-24,2019-10-03,None,None,None,"POLYGON ((-70.60444 42.02472, -70.60444 42.169..."
1,EE,6,104001003D6A7700,18MAY24154658-P1BS-503577655070_01_P001,WV320180524154658P00,DIGITAL GLOBE,WORLDVIEW-3,0,0,2018-05-24,2019-10-03,None,None,None,"POLYGON ((-70.60417 42.02472, -70.60417 42.169..."
2,EE,6,104001003D6A7700,18MAY24154659-M1BS-503577655070_01_P002,WV320180524154659M00,DIGITAL GLOBE,WORLDVIEW-3,1,1,2018-05-24,2019-10-03,None,None,None,"POLYGON ((-70.60389 41.91444, -70.60389 42.069..."
3,EE,6,104001003D6A7700,18MAY24154659-P1BS-503577655070_01_P002,WV320180524154659P00,DIGITAL GLOBE,WORLDVIEW-3,0,0,2018-05-24,2019-10-03,None,None,None,"POLYGON ((-70.60361 41.91444, -70.60361 42.069..."
4,EE,6,104001003D6A7700,18MAY24154701-M1BS-503577655070_01_P003,WV320180524154701M00,DIGITAL GLOBE,WORLDVIEW-3,1,1,2018-05-24,2019-10-03,None,None,None,"POLYGON ((-70.60333 41.80444, -70.60333 41.958..."
5,EE,6,104001003D6A7700,18MAY24154701-P1BS-503577655070_01_P003,WV320180524154701P00,DIGITAL GLOBE,WORLDVIEW-3,0,0,2018-05-24,2019-10-03,None,None,None,"POLYGON ((-70.60306 41.80444, -70.60306 41.958..."
6,EE,6,104001003D6A7700,18MAY24154703-M1BS-503577655070_01_P004,WV320180524154703M00,DIGITAL GLOBE,WORLDVIEW-3,1,1,2018-05-24,2019-10-03,None,None,None,"POLYGON ((-70.60250 41.69444, -70.60250 41.848..."
7,EE,6,104001003D6A7700,18MAY24154703-P1BS-503577655070_01_P004,WV320180524154703P00,DIGITAL GLOBE,WORLDVIEW-3,0,0,2018-05-24,2019-10-03,None,None,None,"POLYGON ((-70.60222 41.69444, -70.60222 41.848..."
8,EE,6,10400100470C8300,19FEB25160014-S2AS_R1C2-504979755010_01_P008,WV320190225160014OR01C0200,DIGITAL GLOBE,WORLDVIEW-3,1,1,2019-02-25,2021-02-15,None,None,None,"POLYGON ((-70.53804 41.69054, -70.53804 41.765..."
9,EE,6,10400100470C8300,19FEB25160015-S2AS_R1C2-504979755010_01_P009,WV320190225160015OR01C0200,DIGITAL GLOBE,WORLDVIEW-3,1,1,2019-02-25,2021-02-15,None,None,None,"POLYGON ((-70.53715 41.75280, -70.53715 41.827..."


### Plot the results

In [11]:
def style_function(hex_value):
    return {'color': hex_value, 'fillOpacity': 0}

# Add OpenStreetMap as a basemap
map = folium.Map()
folium.TileLayer('openstreetmap').add_to(map)

# Create a GeoJson layer from the response_geojson and add it to the map
folium.GeoJson(
    gdf['geometry'].to_json(),
    style_function = lambda x: style_function('#0000FF')
).add_to(map)

# Zoom to collected images
map.fit_bounds(map.get_bounds(), padding=(100, 100))

# Display the map
map

# End